# CureBench Evaluation with Unsloth Models

This notebook demonstrates how to evaluate Unsloth models on the CureBench dataset in Google Colab.

## Overview
- Load Unsloth optimized medical models
- Run evaluation on CureBench dataset  
- Generate competition submission files
- Compatible with Google Colab and GitHub deployment

## 1. Install Required Dependencies

Install Unsloth and other required packages in Google Colab environment.

In [ ]:
# Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" trl peft accelerate bitsandbytes

# Install other required packages
!pip install transformers torch tqdm pandas

## 2. Import Libraries and Check GPU

Import necessary libraries including torch, unsloth, and verify GPU availability in Colab.

In [ ]:
import torch
import gc
import os
from unsloth import FastModel
import pandas as pd
from tqdm import tqdm
import json

# Check GPU availability
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"🔥 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ No GPU detected. Consider using GPU runtime for better performance.")

## 3. Load and Configure the Model

Load the medgemma model with proper configuration for Colab environment, fixing any compatibility issues.

In [ ]:
# Available models - choose one that fits your GPU memory
fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",
    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Medical models
    "unsloth/medgemma-4b-it-bnb-4bit",
] # More models at https://huggingface.co/unsloth

# Choose model based on your needs
model_name = "unsloth/medgemma-4b-it-bnb-4bit"  # Good for medical tasks
# model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit"  # Alternative option

print(f"Loading model: {model_name}")
print("This may take a few minutes...")

# Load model with optimized settings for Colab
model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,
    dtype = None, # None for auto detection
    max_seq_length = 1024, # Choose any for long context!
    load_in_4bit = True,  # Enable 4-bit quantization for memory efficiency
    full_finetuning = False, # We're doing inference only
    # token = "hf_...", # use one if using gated models
)

print("✅ Model loaded successfully!")
print(f"📊 Model name: {model_name}")
print(f"📊 Max sequence length: {model.config.max_position_embeddings if hasattr(model.config, 'max_position_embeddings') else 'Unknown'}")

## 4. Test Model Inference

Test the loaded model with sample prompts to ensure it's working correctly.

In [ ]:
# Test inference with a medical question
test_prompt = """The following is a multiple choice question about medicine. Answer with only the letter (A, B, C, D, or E).

Question: A 65-year-old patient presents with chest pain. Which of the following is the most appropriate initial diagnostic test?
A) CT scan
B) Electrocardiogram (ECG)
C) Blood pressure measurement
D) X-ray
E) MRI

Answer:"""

# Apply chat template
messages = [
    {"role": "system", "content": "You are a helpful medical assistant."},
    {"role": "user", "content": test_prompt}
]

input_text = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# Tokenize and generate
inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False).to(model.device)

print("🧠 Generating response...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# Decode response
response_tokens = outputs[0][inputs.input_ids.shape[-1]:]
response = tokenizer.decode(response_tokens, skip_special_tokens=True)

print("✅ Test inference completed!")
print(f"📝 Question: {test_prompt.split('Question: ')[1].split('Answer:')[0].strip()}")
print(f"🤖 Model Response: {response}")

# Clean up memory
del outputs, inputs
torch.cuda.empty_cache()
gc.collect()

## 5. Clone CureBench Repository and Setup Evaluation

Clone the evaluation framework and prepare for running the benchmark.

In [ ]:
# Clone your GitHub repository (replace with your actual repo URL)
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO_NAME.git

# Navigate to the repository directory
import os
os.chdir('YOUR_REPO_NAME')  # Replace with your repo name

# Install any additional requirements
!pip install -r requirements.txt

print("✅ Repository cloned and dependencies installed!")

## 6. Setup Evaluation Framework

Initialize the CureBench evaluation framework with our Unsloth model.

In [ ]:
# Import the evaluation framework
from eval_framework import CompetitionKit

# Create metadata for submission
metadata = {
    "model_name": model_name,
    "model_type": "UnslothModel", 
    "track": "internal_reasoning",
    "base_model_type": "OpenWeighted",
    "base_model_name": model_name.split('/')[-1],
    "dataset": "cure_bench_phase_1",
    "additional_info": f"Unsloth optimized model with 4-bit quantization running on Colab"
}

# Create dataset configuration
dataset_config = {
    "dataset": {
        "dataset_name": "cure_bench_phase_1",
        "dataset_path": "curebench_testset_phase1.jsonl",  # Update if needed
        "description": "CureBench 2025 Phase 1 test questions"
    },
    "output_dir": "results"
}

# Initialize competition kit
kit = CompetitionKit()
kit.datasets = {"cure_bench_phase_1": dataset_config["dataset"]}

# Load the model in the framework
kit.load_model(
    model_name=model_name,
    model_type="unsloth",
    model_instance=model,
    tokenizer_instance=tokenizer
)

print("✅ Evaluation framework initialized!")
print(f"📊 Model: {model_name}")
print(f"📊 Dataset: cure_bench_phase_1")

## 7. Run Evaluation

Execute the evaluation on the CureBench dataset.

In [ ]:
# Show available datasets
print("Available datasets:")
kit.list_datasets()

# Run evaluation
print("\n🚀 Starting evaluation...")
print("This may take several minutes depending on dataset size.")

results = kit.evaluate("cure_bench_phase_1")

print(f"\n✅ Evaluation completed!")
print(f"📊 Accuracy: {results.accuracy:.2%}")
print(f"📊 Correct predictions: {results.correct_predictions}/{results.total_examples}")
print(f"📊 Total examples processed: {len(results.predictions)}")

## 8. Generate Submission File

Create the competition submission file with proper metadata.

In [ ]:
# Generate submission with metadata
print("📄 Generating submission file...")

submission_filename = f"unsloth_submission_{model_name.split('/')[-1]}.csv"
submission_path = kit.save_submission_with_metadata(
    results=[results],
    metadata=metadata,
    filename=submission_filename
)

print(f"✅ Submission saved to: {submission_path}")
print("📄 Ready for competition submission!")

# Display metadata summary
print("\n📋 Submission metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

# Show some sample predictions
print("\n📝 Sample predictions:")
for i, pred in enumerate(results.predictions[:3]):
    print(f"  Example {i+1}:")
    print(f"    Choice: {pred.get('choice', 'N/A')}")
    print(f"    Answer: {pred.get('open_ended_answer', 'N/A')[:100]}...")
    print()

## 9. Download Results (Optional)

Download the submission file to your local machine.

In [ ]:
# Download the submission file in Google Colab
try:
    from google.colab import files
    print("📥 Downloading submission file...")
    files.download(submission_path)
    print("✅ Download completed!")
except ImportError:
    print("ℹ️ Not running in Google Colab. Submission file is available at:", submission_path)

# List all files in results directory
import os
print("\n📁 Files in results directory:")
for file in os.listdir("results"):
    file_path = os.path.join("results", file)
    file_size = os.path.getsize(file_path) / 1024  # Size in KB
    print(f"  {file} ({file_size:.1f} KB)")

## 10. Cleanup and Memory Management

Free up GPU memory after evaluation.

In [ ]:
# Clean up GPU memory
print("🧹 Cleaning up GPU memory...")

# Delete model and tokenizer
del model, tokenizer

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
# Force garbage collection
gc.collect()

print("✅ Cleanup completed!")
if torch.cuda.is_available():
    print(f"🔥 GPU memory freed: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB available")